Основные библиотеки и используемые методы:
Библиотека scipy и основные статистические функции:http://docs.scipy.org/doc/scipy/reference/stats.html#statistical-functions
Библиотека statmodels для методов коррекции при множественном сравнении:
http://statsmodels.sourceforge.net/devel/stats.html
Статья, в которой рассматриваются примеры использования statsmodels для множественной проверки гипотез:
http://jpktd.blogspot.ru/2013/04/multiple-testing-p-value-corrections-in.html
Описание используемых данных
Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.
В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).
Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.
Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.
В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.
Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.
Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.
Диагноз человека содержится в столбце под названием "Diagnosis".
Практическая значимость изменения
Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:
Fc(C,T)=TC,T>C−CT,T<C
где C,T — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.
Инструкции к решению задачи
Задание состоит из трёх частей. Если не сказано обратное, то уровень значимости нужно принять равным 0.05.


In [72]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint
from statsmodels.sandbox.stats.multicomp import multipletests 

In [73]:
%ls

11.ipynb        banknotes.txt                        hw4.ipynb
12.ipynb        botswana.tsv                         hw5.ipynb
answer_1_1.txt  challenger.txt                       hw6.ipynb
answer_1_2.txt  churn_analysis.csv                   hw7.ipynb
answer_1.txt    churn_analysis.csv~                  hw8.ipynb
answer_2_1.txt  diamonds.txt                         hw9.ipynb
answer_2_2.txt  gene_high_throughput_sequencing.csv  illiteracy.txt
answer_3_1.txt  hw10.ipynb                           pines.txt
answer_3_2.txt  hw1.ipynb                            Untitled.ipynb
AUCs.txt        hw2.ipynb                            water.txt


In [74]:
data = pd.read_csv('gene_high_throughput_sequencing.csv', sep = ',', header = 0)

In [75]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

    - для групп normal (control) и early neoplasia (treatment)
    - для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости. 

In [76]:
data_normal = data[(data.Diagnosis == 'normal')]
data_early = data[(data.Diagnosis == 'early neoplasia')]
data_cancer = data[(data.Diagnosis == 'cancer')]

In [85]:
n = 2; z1 = 0; z2 = 0; p_values_1 = np.array([]); p_values_2 = np.array([])
while (n < data.shape[1]):
    p_values_1 = np.append(p_values_1,stats.ttest_ind(data_normal.ix[:,n], data_early.ix[:,n], equal_var = False)[1])
    p_values_2 = np.append(p_values_2,stats.ttest_ind(data_early.ix[:,n], data_cancer.ix[:,n], equal_var = False)[1])
    if(stats.ttest_ind(data_normal.ix[:,n], data_early.ix[:,n], equal_var = False)[1] < 0.05):
        z1 = z1+1
    if(stats.ttest_ind(data_early.ix[:,n], data_cancer.ix[:,n], equal_var = False)[1] < 0.05):
        z2 = z2+1
    n = n+1

In [86]:
print z1,z2
def save_answer(answer, file_name):
     with open(file_name, "w") as fout:
        fout.write(str(answer))
save_answer(z1, "answer_1_1.txt")
save_answer(z2, "answer_1_2.txt")

1575 3490


Для этой части задания вам понадобится модуль multitest из statsmodels.

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5.

In [96]:
reject, p_corrected, a1, a2 = multipletests(p_values_1, 
                                            alpha = 0.025, 
                                            method = 'holm') 

p_values_1_holm = np.array([])
for n, (a,b) in enumerate(zip(reject, p_corrected)):
    if (a == True):
        p_values_1_holm = np.append(p_values_1_holm, n + 2)
print p_values_1_holm
    

[ 7246.  9822.]


In [97]:
reject, p_corrected, a1, a2 = multipletests(p_values_2, 
                                            alpha = 0.025, 
                                            method = 'holm') 

p_values_2_holm = np.array([])
for n, (a,b) in enumerate(zip(reject, p_corrected)):
    if (a == True):
        p_values_2_holm = np.append(p_values_2_holm, n + 2)
print p_values_2_holm

[    49.    285.    318.   1107.   1240.   1311.   1669.   2035.   2290.
   2291.   2508.   3000.   3033.   3117.   3684.   3905.   4194.   4210.
   4298.   4339.   4425.   4669.   4709.   5317.   5344.   5977.   6412.
   6522.   6647.   7047.   7843.   7898.   8113.   8440.   8717.   8789.
   8816.   8865.   9756.   9788.   9792.   9872.  10015.  10253.  10437.
  10640.  11039.  11113.  11324.  11357.  11844.  11907.  12203.  12213.
  12294.  12295.  12359.  12360.  12441.  12507.  12533.  12967.  13457.
  13521.  13729.  13734.  13818.  13874.  13875.  14191.  14245.  14468.
  14521.  14994.  15092.  15451.  15578.  15642.  15705.]


In [98]:
def fold_change(C, T):
    res = 0
    if T>C:
        res = T/C
    if C>T:
        res = C/T
    return res

In [108]:
z1 = 0
for n in zip(p_values_1_holm):
    if(fold_change(data_normal.ix[:,int(n[0])].mean(),data_early.ix[:,int(n[0])].mean()) >1.5):
        z1 = z1 + 1
        #print fold_change(data_normal.ix[:,int(n[0])].mean(),data_early.ix[:,int(n[0])].mean())
z2= 0        
for n in zip(p_values_2_holm):
    if(fold_change(data_early.ix[:,int(n[0])].mean(),data_cancer.ix[:,int(n[0])].mean()) >1.5):
        #print fold_change(data_early.ix[:,int(n[0])].mean(),data_cancer.ix[:,int(n[0])].mean())
        z2 = z2 + 1
print z1, z2


2 77


In [109]:
save_answer(2, "answer_2_1.txt")
save_answer(77, "answer_2_2.txt")

In [111]:
reject, p_corrected, a1, a2 = multipletests(p_values_1, 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 

p_values_1_bh = np.array([])
for n,t in enumerate(p_corrected):
    if (t < 0.025):
        p_values_1_bh = np.append(p_values_1_bh, n)

for n in zip(p_values_1_bh):
    if(fold_change(np.mean(data_normal.ix[:,int(n[0]) + 2]),np.mean(data_early.ix[:,int(n[0]) + 2 ]))>1.5):
        print fold_change(np.mean(data_normal.ix[:,int(n[0]) + 2 ]),np.mean(data_early.ix[:,int(n[0]) + 2 ]))

1.66382606558
1.50978548204
1.75495105428
1.97486766564


In [112]:
reject, p_corrected, a1, a2 = multipletests(p_values_2, 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 

p_values_2_bh = np.array([])
for n,t in enumerate(p_corrected):
    if (t < 0.025):
        p_values_2_bh = np.append(p_values_2_bh, n)

zbh=0        
for n in zip(p_values_2_bh):
    if(fold_change(np.mean(data_early.ix[:,int(n[0]) + 2]),np.mean(data_cancer.ix[:,int(n[0]) +2]))>1.5):
        #print fold_change(np.mean(data_early.ix[:,int(n[0]) + 2 ]),np.mean(data_cancer.ix[:,int(n[0]) +2]))
        zbh = zbh + 1
print zbh

524


In [113]:
save_answer(4, "answer_3_1.txt")
save_answer(524, "answer_3_2.txt")